## Tool Calling in Agentic AI: Understanding Functions as Tools

In **Agentic AI systems**, a **tool** is a callable **function or capability** that an AI agent can invoke to take action in the external world or perform a specialized computation. Rather than relying only on text generation, an agent decides *when* and *how* to use these tools as part of its reasoning loop.

You can think of tools as items in an **agent’s toolbox**. Just as a human chooses a hammer to drive a nail or a calculator to perform arithmetic, an AI agent selects the appropriate tool when a task requires capabilities beyond pure language understanding.

At the lowest level, most tools are simply **functions**—but functions designed so that an LLM can understand, select, and call them reliably.

## Import the required libraries

In [ ]:
import json
import re
from pydantic import BaseModel
from langchain_ollama import ChatOllama
from langchain.agents import create_agent
from langchain_core.tools import tool, Tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.agents.middleware import AgentMiddleware
from langchain_core.stores import InMemoryStore
from typing import TypedDict, List, Dict, Union, Any


## LLM Setup

In [ ]:
ollama_chat = ChatOllama(
            model="llama3.2:1b",
            temperature=0.0,
            num_ctx=200,
        )

In [ ]:
response = ollama_chat.invoke("What is tool calling in LangChain?")
print(response.content)

## Function

In [ ]:
def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input dictionary or extracts numbers from a string.

    Parameters:
    - inputs (str): 
    string, it should contain numbers that can be extracted and summed.

    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.

    Example Input (Dictionary):
    {"numbers": [10, 20, 30]}

    Example Input (String):
    "Add the numbers 10, 20, and 30."

    Example Output:
    {"result": 60}
    """
    numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]

    result = sum(numbers)
    return {"result": result}

In [ ]:
add_numbers("1 2")

## Tool
A tool is a structured, callable function that an AI agent can invoke to perform a specific action or computation beyond text generation. It enables the agent to interact with external systems, execute logic, and incorporate real-world results into its reasoning.

In [ ]:
add_tool=Tool(
        name="AddTool",
        func=add_numbers,
        description="Adds a list of numbers and returns the result.")

print("tool object",add_tool)

In [ ]:
# Tool name
print("Tool Name:")
print(add_tool.name)

# Tool description
print("\nTool Description:")
print(add_tool.description)

# Tool function
print("\nTool Function:")
print(add_tool.invoke)

In [ ]:
print("Calling Tool Function:")
test_input = "10 25 65 a b" 
print(add_tool.invoke(test_input))  # Example

Testing the tool object ensures:

1. **The tool is correctly set up**:
   - Metadata (`name`, `description`, etc.) is properly defined and aligns with its purpose.
   - The function and schema (if applicable) are correctly configured.

2. **The wrapped function behaves as expected**:
   - The function performs the intended task correctly.
   - It handles edge cases and invalid inputs gracefully.

3. **The tool integrates smoothly with agents**:
   - The tool's output aligns with what the agent expects.
   - There are no compatibility issues when the agent calls the tool.

### `@tool` operator

Now you know how to create a tool with a `Tool` class (using Tool Interface), there's actually another way that you can create a tool using `@tool` decorator. The recommended way to create tools is using the `@tool` decorator. This decorator is designed to simplify the process of tool creation and should be used in most cases. After defining a function, you can decorate it with `@tool` to create a tool that implements the Tool Interface.

`@tool` opertor makes tools out of functions. See below:

In [ ]:
@tool
def add_numbers(inputs:str) -> dict:
    """
    Adds a list of numbers provided in the input string.
    Parameters:
    - inputs (str): 
    string, it should contain numbers that can be extracted and summed.
    Returns:
    - dict: A dictionary with a single key "result" containing the sum of the numbers.
    Example Input:
    "Add the numbers 10, 20, and 30."
    Example Output:
    {"result": 60}
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    # numbers = [int(x) for x in inputs.replace(",", "").split() if x.isdigit()]
    
    result = sum(numbers)
    return {"result": result}

In [ ]:
print("Name: \n", add_numbers.name)
print("\nDescription: \n", add_numbers.description) 
print("\nArgs: \n", add_numbers.args) 

In [ ]:
test_input = "what is the sum between 10, 20 and 30 " 
print(add_numbers.invoke(test_input))  # Example


### Use @tool-StructuredTool

The @tool decorator creates a StructuredTool with schema information extracted from function signatures and docstrings as show here. This helps LLMs better understand what inputs the tool expects and how to use it properly. While both approaches work, @tool is generally preferred for modern LangChain applications, especially with LangGraph and function-calling models.

In [ ]:
# Comparing the two approaches
print("Tool Constructor Approach:")

print(f"Has Schema: {hasattr(add_tool, 'args_schema')}")
print("\n")

print("@tool Decorator Approach:")


print(f"Has Schema: {hasattr(add_numbers, 'args_schema')}")
print(f"Args Schema Info: {add_numbers.args}")


In this example, the tool has two inputs: a string containing the numbers to add, and a second boolean input that determines whether to sum the absolute values of those numbers.


In [ ]:
@tool
def add_numbers_with_options(numbers: List[float], absolute: bool = False) -> float:
    """
    Adds a list of numbers provided as input.

    Parameters:
    - numbers (List[float]): A list of numbers to be summed.
    - absolute (bool): If True, use the absolute values of the numbers before summing.

    Returns:
    - float: The total sum of the numbers.
    """
    if absolute:
        numbers = [abs(n) for n in numbers]
    return sum(numbers)

Let's compare the arguments for add_numbers_with_options and add_numbers. Both are structured tools. They both include the inputs field, which is a string input. However, add_numbers_with_options has an additional key-value pair: absolute, a boolean field with a default value of False. This means add_numbers_with_options supports optional behavior—taking the absolute value of the numbers—while add_numbers only handles basic numeric extraction and summation


In [ ]:
print(f"Args Schema Info: {add_numbers_with_options.args}")
print(f"Args Schema Info: {add_numbers.args}")

In [ ]:
print(add_numbers_with_options.invoke({"numbers":[-1.1,-2.1,-3.0],"absolute":False}))
print(add_numbers_with_options.invoke({"numbers":[-1.1,-2.1,-3.0],"absolute":True}))

## Improved tool return types with Python typing

When creating tools, you must accurately specify their return values. This helps the agent understand and handle different possible outputs.



The function `sum_numbers_with_complex_output` returns a more flexible output format. It returns a dictionary containing a float value when numbers are successfully summed, or a descriptive error message as a string if no numbers are found or an issue occurs during processing.

In [ ]:
@tool
def sum_numbers_with_complex_output(inputs: str) -> Dict[str, Union[float, str]]:
    """
    Extracts and sums all integers and decimal numbers from the input string.

    Parameters:
    - inputs (str): A string that may contain numeric values.

    Returns:
    - dict: A dictionary with the key "result". If numbers are found, the value is their sum (float). 
            If no numbers are found or an error occurs, the value is a corresponding message (str).

    Example Input:
    "Add 10, 20.5, and -3."

    Example Output:
    {"result": 27.5}
    """
    matches = re.findall(r'-?\d+(?:\.\d+)?', inputs)
    if not matches:
        return {"result": "No numbers found in input."}
    try:
        numbers = [float(num) for num in matches]
        total = sum(numbers)
        return {"result": total}
    except Exception as e:
        return {"result": f"Error during summation: {str(e)}"}

The function `sum_numbers_from_text` returns a straightforward output format. It extracts all integer values from the input string, sums them, and returns the total as a float. This function assumes that at least one valid number is present in the input and does not handle cases where no numbers are found or where an error might occur.


In [ ]:
@tool
def sum_numbers_from_text(inputs: str) -> float:
    """
    Adds a list of numbers provided in the input string.
    
    Args:
        text: A string containing numbers that should be extracted and summed.
        
    Returns:
        The sum of all numbers found in the input.
    """
    # Use regular expressions to extract all numbers from the input
    numbers = [int(num) for num in re.findall(r'\d+', inputs)]
    result = sum(numbers)
    return result

## Agent

In [ ]:
agent = create_agent(
    model=ollama_chat,
    tools=[add_numbers],
    # Optional: system_prompt="You are a helpful assistant."
)

In [ ]:
def run_math_agent(agent, user_input: str):
    response = agent.invoke(
        {"messages": [{"role": "user", "content": user_input}]}
    )

    final_msg = response["messages"][-1]

    return {
        "input": user_input,
        "output": final_msg.content
    }

In [ ]:
run_math_agent(agent, user_input= "Add 10, 20, two and 30")

## `create_agent`

`create_agent` is the **modern replacement** for `initialize_agent` in LangChain.
It creates an **agent as a stateful execution graph** that repeatedly reasons with an LLM and calls tools until a stopping condition is reached.

Instead of a hidden loop, the agent is now an **explicit, inspectable graph**, which makes it more reliable and production-ready.

---

### **Relationship between agent and LLM**

* The **agent** is an execution controller implemented as a **state graph**.

  * It manages the conversation state.
  * It decides when to call tools.
  * It controls the loop (model → tool → model).
* The **LLM** is the reasoning engine.

  * Interprets user messages.
  * Decides whether a tool is needed.
  * Produces tool calls or final answers.

Think of the agent as a **workflow engine**, and the LLM as the **decision-making brain** inside that workflow.

---

### **Conceptual Mapping (Old → New)**

| Old (`initialize_agent`) | New (`create_agent`)    |
| ------------------------ | ----------------------- |
| Hidden loop              | Explicit state graph    |
| `agent` enum             | Built-in reasoning loop |
| `verbose=True`           | `debug=True`            |
| Implicit memory          | Checkpointer + state    |
| Classic agents           | LangGraph-based agents  |

---


### **Key parameters of `create_agent`**

#### **1. `model`**

* The language model used for reasoning.
* Can be:

  * A string identifier (e.g. `"openai:gpt-4o-mini"`)
  * A chat model instance (e.g. `ChatOpenAI`)
* The model must support **tool calling** for agentic behavior.

##### MISTRAL

For this agent, we use **`sum_numbers_with_complex_output`** as the tool and **`mistral` (via Ollama)** as the LLM, since it supports native tool calling and structured outputs.
Smaller models like **`llama3.2:1b`** lack reliable tool-calling and cannot correctly parse dictionary-based tool responses.
As a result, such models often produce parsing or validation errors when used with tools that return structured data.

In [ ]:
mistral_llm = ChatOllama(
    model="mistral:latest",
    temperature=0,
    num_ctx= 300
)

#### **2. `tools`**

A tool is a structured, callable function that an AI agent can invoke to perform a specific action or computation beyond text generation. It enables the agent to interact with external systems, execute logic, and incorporate real-world results into its reasoning.

* A list of tools the agent can call.
* Tools can be:

  * `@tool`-decorated functions
  * `BaseTool` instances
  * Callable functions with schemas
* If `None` or empty:

  * The agent behaves like a plain chat model (no tool loop).

Tools represent the agent’s **action space**.

#### **3. `system_prompt`**

* Sets the agent’s high-level behavior and identity.
* Added as a `SystemMessage` at the start of every interaction.
* Used to guide:

  * Tool usage
  * Tone
  * Constraints
  * Role (e.g., “You are a math-solving agent”)

This replaces the old implicit agent instructions.

In [ ]:
systemprompt = SystemMessage(
    content=("You are a math agent. Always use tools for calculations. "
             "Respond ONLY in valid JSON with keys: input, tool_called, output."
            ))

#### **4. Tool-calling loop (implicit)**

Unlike `initialize_agent`, there is **no `agent` parameter** such as `"zero-shot-react-description"`.

Instead:

* The **ReAct-style loop** is built-in:

  * **Reason** → Model thinks
  * **Act** → Tool is called (if needed)
  * **Observe** → Tool output is added to state
  * **Repeat** → Until no more tools are required

This loop is implemented internally using **LangGraph**.

#### **5. `middleware`**

* Allows interception and modification of agent behavior.
* Can be used for:

  * Logging
  * Validation
  * Custom control logic
* Middleware operates at specific hooks in the agent lifecycle.

Think of middleware as **plugins for agent behavior**.

In [ ]:
class LoggingMiddleware(AgentMiddleware):
    def on_model_start(self, state, context):
        print("🔍 Model is reasoning...")
        return state

#### **6. `response_format`**

* Enables **structured outputs**.
* Can be:

  * A Pydantic model
  * A structured response strategy
* Ensures the agent produces outputs in a predictable schema.

Useful for APIs, automation, and production systems.

In [ ]:
class AgentJSONResponse(BaseModel):
    input: str
    tool_called: str
    output: str

#### **7. `state_schema`**

* Extends the default `AgentState`.
* Lets you add custom state fields without rewriting the agent.
* Useful when the agent needs memory beyond messages and tool results.

In [ ]:
class MyAgentState(TypedDict):
    messages: List[Any]
    last_tool_used: str | None

#### **8. `checkpointer`**

* Enables persistence of agent state.
* Common use cases:

  * Chat memory
  * Conversation recovery
  * Long-running agents

In [ ]:
store = InMemoryStore()

#### **9. `interrupt_before` / `interrupt_after`**

* Allows pausing execution:

  * Before a tool is called
  * After a node runs
* Useful for:

  * Human-in-the-loop confirmation
  * Safety checks
  * Debugging


#### **10. `debug`**

* If `True`, prints detailed execution logs:

  * Node transitions
  * State updates
  * Tool calls
* Replaces `verbose=True` from old agents.

In [ ]:
class MyContext(TypedDict):
    user_id: str

### Math Agent

#### **What `create_agent` returns**

* Returns a **CompiledStateGraph**
* This graph:

  * Maintains agent state
  * Executes tool calls
  * Can be composed into larger multi-agent systems
* Invoked using `.invoke()` or `.stream()`

In [ ]:
math_agent = create_agent(
    model=mistral_llm,
    tools=[add_numbers],
    system_prompt=systemprompt,
    response_format=AgentJSONResponse,
    state_schema=MyAgentState,
    context_schema=MyContext,
    store=store,
    debug=False,
    name="math_agent"
)

In [ ]:
result = run_math_agent(math_agent, "Add 10, 20 and 30")
result

In [ ]:
query = """In 2023, the US GDP was approximately $27.72 trillion, 
while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion. What is the total?"""
result = run_math_agent(math_agent, query)
result

### Math Agent 2

In [ ]:
# math_agent_2 using sum_numbers_from_text
math_agent_2 = create_agent(
    model=mistral_llm,           # Mistral LLM (Ollama)
    tools=[sum_numbers_from_text], # Tool for parsing & summing numbers from text
    debug=True,    # Set True to see agent reasoning
    name="math_agent_2"
)

In [ ]:
query =  "Add 10, 20 and 30"

result = run_math_agent(math_agent_2, query)
result

### Math Agent 3

In [ ]:
# math_agent_3 using sum_numbers_with_complex_output
math_agent_3 = create_agent(
    model=mistral_llm,  # Your Mistral LLM
    tools=[sum_numbers_with_complex_output],  # Tool that returns complex/structured output
    debug=True,    # Set True to see reasoning steps
    name="math_agent_3"
)

In [ ]:
query =  "Add 10, 20 and 30"

result = run_math_agent(math_agent_3, query)
result

### Math Agent 4

In [ ]:
math_agent_4 = create_agent(
    model=mistral_llm,  # Your Mistral LLM
    tools=[add_numbers_with_options],  # Tool that returns complex/structured output
    debug=True,    # Set True to see reasoning steps
    name="math_agent_4"
)

In [ ]:
query =  "Add -10, -20, and -30 using absolute values."

result = run_math_agent(math_agent_4, query)
result

## Orchestrating multiple tools with an agent: Mathematical toolkit

In [ ]:
@tool
def subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string, negates the first number, and successively subtracts 
    the remaining numbers in the list.

    This function is designed to handle input in string format, where numbers are separated 
    by spaces, commas, or other delimiters. It parses the string, extracts valid numeric values, 
    and performs a step-by-step subtraction operation starting with the first number negated.

    Parameters:
    - inputs (str): 
      A string containing numbers to subtract. The string may include spaces, commas, or 
      other delimiters between the numbers.

    Returns:
    - dict: 
      A dictionary containing the key "result" with the calculated difference as its value. 
      If no valid numbers are found in the input string, the result defaults to 0.

    Example Input:
    "100, 20, 10"

    Example Output:
    {"result": -130}

    Notes:
    - Non-numeric characters in the input are ignored.
    - If the input string contains only one valid number, the result will be that number negated.
    - Handles a variety of delimiters (e.g., spaces, commas) but does not validate input formats 
      beyond extracting numeric values.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Start with the first number negated
    result = -1 * numbers[0]

    # Subtract all subsequent numbers
    for num in numbers[1:]:
        result -= num

    return {"result": result}

In [ ]:
print("Name: \n", subtract_numbers.name)
print("Description: \n", subtract_numbers.description) 
print("Args: \n", subtract_numbers.args)

In [ ]:
print("Calling Tool Function:")
test_input = "10 20 30 and four a b" 
print(subtract_numbers.invoke(test_input))  # Example

In [ ]:
# Multiplication Tool
@tool
def multiply_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates their product.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the product of the numbers.

    Example Input:
    "2, 3, 4"

    Example Output:
    {"result": 24}

    Notes:
    - If no numbers are found, the result defaults to 1 (neutral element for multiplication).
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]
    print(numbers)

    # If no numbers are found, return 1
    if not numbers:
        return {"result": 1}

    # Calculate the product of the numbers
    result = 1
    for num in numbers:
        result *= num
        print(num)

    return {"result": result}

In [ ]:
# Division Tool
@tool
def divide_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and calculates the result of dividing the first number 
    by the subsequent numbers in sequence.

    Parameters:
    - inputs (str): A string containing numbers separated by spaces, commas, or other delimiters.

    Returns:
    - dict: A dictionary with the key "result" containing the quotient.

    Example Input:
    "100, 5, 2"

    Example Output:
    {"result": 10.0}

    Notes:
    - If no numbers are found, the result defaults to 0.
    - Division by zero will raise an error.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]


    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Calculate the result of dividing the first number by subsequent numbers
    result = numbers[0]
    for num in numbers[1:]:
        result /= num

    return {"result": result}

In [ ]:
# Testing multiply_tool
multiply_test_input = "2, 3, and four "
multiply_result = multiply_numbers.invoke(multiply_test_input)
print("--- Testing MultiplyTool ---")
print(f"Input: {multiply_test_input}")
print(f"Output: {multiply_result}")

In [ ]:
# Testing divide_tool
divide_test_input = "100, 5, two"
divide_result = divide_numbers.invoke(divide_test_input)
print("--- Testing DivideTool ---")
print(f"Input: {divide_test_input}")
print(f"Output: {divide_result}")

In [ ]:
@tool
def calculate_power(input_text: str) -> dict:
    """
    Calculates the power of a number (x^y).

    Parameters:
    - input_text (str): A string like "2, 3", "2 3", "5^2", or "2 to the power of 3".

    Returns:
    - dict: {"result": <calculated value>} or an error message.
    """
    # Try to extract expressions like "5^2"
    match = re.search(r"(\d+(?:\.\d+)?)\s*\^+\s*(\d+(?:\.\d+)?)", input_text)
    if match:
        base = float(match.group(1))
        exponent = float(match.group(2))
        return {"result": base ** exponent}

    # Try to extract expressions like "2 to the power of 3"
    match = re.search(r"(\d+(?:\.\d+)?)\s*(?:to\s+the\s+power\s+of)\s*(\d+(?:\.\d+)?)", input_text, re.IGNORECASE)
    if match:
        base = float(match.group(1))
        exponent = float(match.group(2))
        return {"result": base ** exponent}

    # Fallback: assume two numbers separated by space or comma
    try:
        numbers = [float(num) for num in input_text.replace(",", " ").split()]
        if len(numbers) != 2:
            return {"result": "Invalid input. Please provide exactly two numbers."}
        base, exponent = numbers
        return {"result": base ** exponent}
    except ValueError:
        return {"result": "Invalid input format. Provide input like '2 3', '2^3', or '2 to the power of 3'."}


In [ ]:
# Testing calculate_power
power_test_input = "2, 5"
power_result = calculate_power.invoke(power_test_input)
print("--- Testing PowerTool ---")
print(f"Input: {power_test_input}")
print(f"Output: {power_result}")

In [ ]:
tools = [add_numbers,subtract_numbers, multiply_numbers, divide_numbers, calculate_power]
tools

In [ ]:
sys_prompt="You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."

In [ ]:
maths_agent = create_agent(
    model=mistral_llm,
    tools=tools,
    system_prompt=sys_prompt,
    name="maths_agent"
)

In [ ]:
query = "What is 25 divided by 4?"
res = run_math_agent(maths_agent,query)
res

In [ ]:
query = "Subtract 100, 20, and 10."
res = run_math_agent(maths_agent,query)
res

In [ ]:
query = "Calculate 2 to the power of 5."
res = run_math_agent(maths_agent,query)
res

In [ ]:
@tool
def new_subtract_numbers(inputs: str) -> dict:
    """
    Extracts numbers from a string and performs subtraction sequentially, starting with the first number.

    This function is designed to handle input in string format, where numbers may be separated by spaces, 
    commas, or other delimiters. It parses the input string, extracts numeric values, and calculates 
    the result by subtracting each subsequent number from the first. inputs[0]-inputs[1]-inputs[2]

    Parameters:
    - inputs (str): 
      A string containing numbers to subtract. The string can include spaces, commas, or other 
      delimiters between the numbers.

    Returns:
    - dict: 
      A dictionary containing the key "result" with the calculated difference as its value. 
      If no valid numbers are found in the input string, the result defaults to 0.

    Example Usage:
    - Input: "100, 20, 10"
    - Output: {"result": 70}

    Limitations:
    - The function does not handle cases where numbers are formatted with decimals or other non-integer representations.
    """
    # Extract numbers from the string
    numbers = [int(num) for num in inputs.replace(",", "").split() if num.isdigit()]

    # If no numbers are found, return 0
    if not numbers:
        return {"result": 0}

    # Start with the first number
    result = numbers[0]

    # Subtract all subsequent numbers
    for num in numbers[1:]:
        result -= num

    return {"result": result}

In [ ]:
tools_updated = [add_numbers, new_subtract_numbers, multiply_numbers, divide_numbers]

#sys_prompt=
sys_prompt = SystemMessage(
    content=("You are a helpful mathematical assistant that can perform various operations. Use the tools precisely and explain your reasoning clearly."))

maths_agent_new = create_agent(
    model=mistral_llm,
    tools=tools_updated,
    system_prompt=sys_prompt,
    state_schema=MyAgentState,
    context_schema=MyContext,
    store=store,
    #debug=True,
    name="maths_agent_new"
)

In [ ]:
query = "Multiply 2, 3, and 4." #"Subtract 100, 20, and 10."

res = run_math_agent(maths_agent_new,query)
res

In [ ]:
TOOLS = {
    "add_numbers": ["add_numbers"],
    "new_subtract_numbers": ["subtract_numbers"],
    "multiply_numbers": ["multiply_numbers"],
    "divide_numbers": ["divide_numbers"]
}

def extract_tool_and_result(output_text: str, tools=TOOLS):
    tool_called = None
    result = None

    if not output_text or not output_text.strip():
        return None, None

    # 1️⃣ Detect tool ONLY from function definition
    for canonical_tool, fn_names in tools.items():
        for fn in fn_names:
            pattern = rf"def\s+{re.escape(fn)}\s*\("
            if re.search(pattern, output_text):
                tool_called = canonical_tool
                break
        if tool_called:
            break

    # 2️⃣ Extract result from JSON-like output
    json_match = re.search(
        r"[\"']result[\"']\s*:\s*([0-9]+(?:\.[0-9]+)?)",
        output_text
    )
    if json_match:
        result = float(json_match.group(1))
        return tool_called, result

    # 3️⃣ Fallback: extract from explanation text (e.g. "equals 70")
    text_match = re.search(
        r"(equals|is)\s+([0-9]+(?:\.[0-9]+)?)",
        output_text
    )
    if text_match:
        result = float(text_match.group(2))

    return tool_called, result

def infer_expected_tool(query: str):
    q = query.lower()
    if "add" in q or "sum" in q:
        return "add_numbers"
    if "subtract" in q:
        return "new_subtract_numbers"
    if "multiply" in q:
        return "multiply_numbers"
    if "divide" in q:
        return "divide_numbers"
    return None

In [ ]:
output_text = res["output"]
extract_tool_and_result(output_text), infer_expected_tool(query)

In [ ]:
# Test Cases
test_cases = [
    {
        "query": "Subtract 100, 20, and 10.",
        "expected": {"result": 70},
        "description": "Testing subtraction tool with sequential subtraction."
    },
    {
        "query": "Multiply 2, 3, and 4.",
        "expected": {"result": 24},
        "description": "Testing multiplication tool for a list of numbers."
    },
    {
        "query": "Divide 100 by 5 and then by 2.",
        "expected": {"result": 10.0},
        "description": "Testing division tool with sequential division."
    },
    {
        "query": "Subtract 50 from 20.",
        "expected": {"result": -30},
        "description": "Testing subtraction tool with negative results."
    }

]

In [ ]:
correct_tasks = []

for index, test in enumerate(test_cases, start=1):
    query = test["query"]
    expected_result = test["expected"]["result"]
    expected_tool = infer_expected_tool(query)

    print(f"\n--- Test Case {index}: {test['description']} ---")
    print(f"Query: {query}")

    response = run_math_agent(maths_agent_new, query)
    output_text = response["output"]

    tool_called, tool_result = extract_tool_and_result(output_text)

    print(f"Expected Tool: {expected_tool}")
    print(f"Detected Tool: {tool_called}")
    print(f"Expected Result: {expected_result}")
    print(f"Detected Result: {tool_result}")

    if tool_called == expected_tool and tool_result == expected_result:
        print(f"✅ Test Passed: {test['description']}")
        correct_tasks.append(test["description"])
    else:
        print(f"❌ Test Failed: {test['description']}")

## **Exploring LangChain's built-in tools**

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper

# Create a Wikipedia tool using the @tool decorator
@tool
def search_wikipedia(query: str) -> str:
    """Search Wikipedia for factual information about a topic.
    
    Parameters:
    - query (str): The topic or question to search for on Wikipedia
    
    Returns:
    - str: A summary of relevant information from Wikipedia
    """
    wiki = WikipediaAPIWrapper()
    return wiki.run(query)

In [ ]:
search_wikipedia.invoke("What is tool calling?")

In [ ]:
# Update your tools list to include the Wikipedia tool
tools_updated = [add_numbers, new_subtract_numbers, multiply_numbers, divide_numbers, search_wikipedia]

#sys_prompt=
sys_prompt = SystemMessage(
    content=(
        "You are a helpful assistant that can perform various mathematical operations and look up information. Use the tools precisely and explain your reasoning clearly."))

math_agent_updated = create_agent(
    model=mistral_llm,
    tools=tools_updated,
    system_prompt=sys_prompt,
    state_schema=MyAgentState,
    context_schema=MyContext,
    store=store,
    #debug=True,
    name="math_agent_updated"
)

In [ ]:
query = "What is the population of Canada? Multiply it by 0.75"

# response = run_math_agent(math_agent_updated, query)
# response

raw_response = math_agent_updated.invoke(
    {"messages": [{"role": "user", "content": query}]}
)


In [ ]:
response

In [ ]:

print("\nMessage sequence:")
for i, msg in enumerate(raw_response["messages"]):
    print(f"\n--- Message {i+1} ---")
    print(f"Type: {type(msg).__name__}")
    if hasattr(msg, "content"):
        print(f"Content: {msg.content}")
    if hasattr(msg, "name"):
        print(f"Name: {msg.name}")
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        print(f"Tool calls: {msg.tool_calls}")
